# 価格予測モデルのBaseline  
- CNNを用いたモデルを作成する．  
- 価格予測とクラス分類でタスクが大きく異なるので，imagenetで学習したモデルを用いないものを最初に作成する．  
- サイトに載せられる画像を教師データとしており，画像が大きく回転したりなどは不要と考えられるためそのような前処理は行わない．  
- 損失関数にはmaeもしくはrmseを用いる．  

## モデルの構築  
- EfficientNetB0（未学習）を用いて特徴量を抽出．  
- num_sales, コレクション名のone-hotベクトルを抽出した特徴量に結合．  
- 全結合層を重ねて出力．  
- ImageNetを用いて事前学習したものとしていないもので比較する．  
- 目的変数をそのまま予測するとスケールが大きすぎるので，先に対数変換して評価関数にRMSE, MAEなどを用いるほうが良いかも．  
- **このノートブックでやっているのは事前学習有り．**  

## 評価関数  
- RMSLEを用いる．  
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n (\log{(y_i+1)} - \log{(\hat{y_i} +1)})^2}$$

- 追加でMAPEを用いてみる．  
$$MAPE = \frac{100}{n} \sum_{i=1}^n |\frac{\hat{y}_i - y_i}{y_i}|$$

タスクAに関してはデータ不足の可能性が考えられるため，特徴量抽出とともにデータを追加で収集する．  

## 変数（タスクA）  
- 目的変数: last_sale.total_price  
- 説明変数: 画像データ，コレクション名（collection.name），num_sales，

## 変数（タスクB）  
- 目的変数: last_sale.total_price  
- 説明変数: 画像データ  

In [1]:
import os
from typing import List, Optional, Tuple, Dict
import math
import tempfile
import random

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import cv2
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optim
import tensorflow.keras.activations as activations
from tensorflow.keras.utils import Sequence
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras.callbacks as callbacks
from tensorflow.keras.applications import EfficientNetB0 as efn
import cloudpickle

In [2]:
A_IMGPATH = "../data/taskA/img"
A_DFPATH = "../data/taskA/table"
B_IMGPATH = "../data/taskB/img"
B_DFPATH = "../data/taskB/table"
asset_df_A = pd.read_csv(os.path.join(A_DFPATH, "asset_data.csv"))
asset_df_B = pd.read_csv(os.path.join(B_DFPATH, "asset_data.csv"))

asset_df_A = asset_df_A.rename(columns={"last_sale.total_price": "target"})
asset_df_B = asset_df_B.rename(columns={"last_sale.total_price": "target"})

asset_df_A = pd.concat((asset_df_A, pd.get_dummies(asset_df_A["collection.name"])), axis=1)
asset_df_B[asset_df_A["collection.name"].unique()] = 0

asset_df_A["full_path"] =\
    asset_df_A["image_id"].apply(lambda x: A_IMGPATH + "/" + x)
asset_df_B["full_path"] =\
    asset_df_B["image_id"].apply(lambda x: B_IMGPATH + "/" + x)

asset_df_A['target'] = asset_df_A['target'].astype(float) * 1e-18
asset_df_B['target'] = asset_df_B['target'].astype(float) * 1e-18
asset_df_A = asset_df_A.query('target > 0')
asset_df_B = asset_df_B.query('target > 0')
asset_df_A['target'] = asset_df_A['target'].apply(lambda x: np.log1p(x))
asset_df_B['target'] = asset_df_B['target'].apply(lambda x: np.log1p(x))

os.makedirs("../models", exist_ok=True)

print(f"data shape: {asset_df_A.shape}")
print(f"data shape: {asset_df_B.shape}")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,27,28,71,88,119) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (121,122,123,124) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


data shape: (21747, 169)
data shape: (5200, 177)


## Helper Functions  

### DataLoader  

In [3]:
class FullPathDataLoader(Sequence):
    """
    Data loader that load images, meta data and targets.
    This class is inherited Sequence class of Keras.
    """

    def __init__(self, path_list: np.ndarray, target: Optional[np.ndarray],
                 meta_data: Optional[np.ndarray] = None, batch_size: int = 16,
                 task: str = "B", width: int = 256, height: int = 256,
                 resize: bool = True, shuffle: bool = True, is_train: bool = True):
        """
        Constructor. This method determines class variables.

        Parameters
        ----------
        path_list : np.ndarray[str]
            The array of absolute paths of images.
        meta_data : np.ndarray[int]
            One-hot vector of collections.
        target : np.ndarray
            Array of target variavles.
        batch_size : int
            Batch size used when model training.
        task : str
            Please determine this data loader will be used for task A or B(default=A).
        width : int
            Width of resized image.
        height : int
            Height of resize image.
        resize : bool
            Flag determine whether to resize.
        shuffle : bool
            Flag determine whether to shuffle on epoch end.
        is_train : bool
            Determine whether this data loader will be used training model.
            if you won't this data loader, you have set 'is_train'=False.
        """
        self.path_list = path_list
        self.batch_size = batch_size
        self.task = task
        self.width = width
        self.height = height
        self.resize = resize
        self.shuffle = shuffle
        self.is_train = is_train
        self.length = math.ceil(len(self.path_list) / self.batch_size)

        if self.is_train:
            self.target = target
        if self.task == "A":
            self.meta_data = meta_data

    def __len__(self):
        """
        Returns
        -------
        self.length : data length
        """
        return self.length

    def get_img(self, path_list: np.ndarray):
        """
        Load image data and resize image if 'resize'=True.

        Parameters
        ----------
        path_liist : np.ndarray
            The array of relative image paths from directory 'dir_name'.
            Size of this array is 'batch_size'.

        Returns
        -------
        img_list : np.ndarray
            The array of image data.
            Size of an image is (width, height, 3) if 'resize'=True.
        '"""
        img_list = []
        for path in path_list:
            img = cv2.imread(path)
            img = cv2.resize(img, (self.width, self.height))
            img = img / 255.
            img_list.append(img)

        img_list = np.array(img_list)
        return img_list

    def _shuffle(self):
        """
        Shuffle path_list, meta model.
        If 'is_train' is True, target is shuffled in association path_list.
        """
        idx = np.random.permutation(len(self.path_list))
        self.path_list = self.path_list[idx]
        if self.task == "A":
            self.meta_data = self.meta_data[idx]
        if self.is_train:
            self.target = self.target[idx]

    def __getitem__(self, idx):
        path_list = self.path_list[self.batch_size*idx:self.batch_size*(idx+1)]
        img_list = self.get_img(path_list)
        if self.is_train:
            target_list = self.target[self.batch_size*idx:self.batch_size*(idx+1)]
            if self.task == "A":
                meta = self.meta_data[self.batch_size*idx:self.batch_size*(idx+1)]
                return (img_list, meta), target_list
            else:
                return img_list, target_list
        else:
            if self.task == "A":
                meta = self.meta_data[self.batch_size*idx:self.batch_size*(idx+1)]
                return ((img_list, meta),)
            else:
                return img_list

    def on_epoch_end(self):
        if self.is_train:
            self._shuffle()

### seed settings  

In [4]:
def set_seed(random_state=6174):
    tf.random.set_seed(random_state)
    np.random.seed(random_state)
    random.seed(random_state)
    os.environ['PYTHONHASHSEED'] = str(random_state)

### Create model  

In [5]:
def create_model(input_shape: Tuple[int], output_shape: int,
                 activation, loss, meta_shape: Optional[int] = None,
                 task: str = "B", learning_rate: float = 0.001,
                 pretrain: bool = False) -> models.Model:
    """
    The function for creating model.

    Parameters
    ----------
    input_shape : int
        Shape of input image data.
    output_shape : int
        Shape of model output.
    activation : function
        The activation function used hidden layers.
    loss : function
        The loss function of model.
    meta_shape : int
        Shape of input meta data of image.
    task : str
        Please determine this model will be used for task A or B(default=A).
    learning_rate : float
        The learning rate of model.
    pretrain : bool
        Flag that deterimine whether use pretrain model(default=False).

    Returns
    -------
    model : keras.models.Model
        Model instance.
    """
    if pretrain:
        weights = 'imagenet'
    else:
        weights = None

    inputs = layers.Input(shape=input_shape)
    efn_model = efn(include_top=False, input_shape=input_shape,
                    weights=weights)(inputs)
    ga = layers.GlobalAveragePooling2D()(efn_model)

    if task == "A":
        meta_inputs = layers.Input(shape=meta_shape)
        concate = layers.Concatenate()([ga, meta_inputs])
        dense1 = layers.Dense(units=128)(concate)
        av1 = layers.Activation(activation)(dense1)
        dr1 = layers.Dropout(0.3)(av1)
        dense2 = layers.Dense(units=64)(dr1)
        av2 = layers.Activation(activation)(dense2)
        dr2 = layers.Dropout(0.3)(av2)
        outputs = layers.Dense(output_shape)(dr2)

        model = models.Model(inputs=[inputs, meta_inputs], outputs=[outputs])

    elif task == "B":
        dense1 = layers.Dense(units=128)(ga)
        av1 = layers.Activation(activation)(dense1)
        dr1 = layers.Dropout(0.3)(av1)
        dense2 = layers.Dense(units=64)(dr1)
        av2 = layers.Activation(activation)(dense2)
        dr2 = layers.Dropout(0.3)(av2)
        outputs = layers.Dense(output_shape)(dr2)

        model = models.Model(inputs=[inputs], outputs=[outputs])

    else:
        raise Exception("Please set task is A or B.")

    model.compile(loss=loss,
                  optimizer=optim.SGD(learning_rate=learning_rate, momentum=0.9),
                  metrics=['mae', 'mse'])
    return model

### Training model  

In [6]:
def train(path_list: np.ndarray, target: np.ndarray, loss,
          meta_data: Optional[np.ndarray] = None, task: str = "B"):
    """
    The function for training model.

    Parameters
    ----------
    path_list : np.ndarray
        The path list of all image data.
    target : np.ndarray
        The array of targets data.
    loss : function
        The loss function of keras.
    meta_data : np.ndarray
        The array of meta data of image.
    task : str
        Please determine you train model for task A or B(default=A).
    """
    if task == "A":
        train_path, val_path, train_meta, val_meta, train_y, val_y =\
            train_test_split(path_list, meta_data, target, test_size=0.1, random_state=6174)
        train_gen = FullPathDataLoader(path_list=train_path, target=train_y,
                                       meta_data=train_meta, batch_size=16,
                                       task=task)
        val_gen = FullPathDataLoader(path_list=val_path, target=train_y,
                                     meta_data=val_meta, batch_size=1,
                                     task=task)
    elif task == "B":
        train_path, val_path, train_y, val_y =\
            train_test_split(path_list, target, test_size=0.1, random_state=6174)
        train_gen = FullPathDataLoader(path_list=train_path, target=train_y,
                                       batch_size=16, task=task)
        val_gen = FullPathDataLoader(path_list=val_path, target=val_y,
                                     batch_size=1, task=task)
    else:
        raise Exception("Please set task is A or B")

    set_seed()
    model = NFTModel(
        create_model(input_shape=(256, 256, 3), output_shape=1,
                     activation=activations.relu, loss=loss,
                     meta_shape=len(meta_features), task=task,
                     learning_rate=0.00001, pretrain=True)
    )

    ES = callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                 restore_best_weights=True)

    print("starting training")
    print('*' + '-' * 30 + '*')

    model.fit(train_gen, val_gen, epochs=100, batch_size=16,
              callbacks=[ES])

    print("finished training")
    print('*' + '-' * 30 + '*' + '\n')

    if task == "A":
        val_gen = FullPathDataLoader(path_list=val_path, target=train_y,
                                     meta_data=val_meta, batch_size=1, task=task,
                                     shuffle=False, is_train=False)
    else:
        val_gen = FullPathDataLoader(path_list=val_path, target=train_y,
                                     batch_size=1, task=task,
                                     shuffle=False, is_train=False)
    print("starting evaluate")
    print('*' + '-' * 30 + '*')

    model.evaluate(val_gen, val_y)

    print("finished evaluate")
    print('*' + '-' * 30 + '*' + '\n')

    return model

In [7]:
class NFTModel(KerasRegressor):
    """
    Model class.
    This class is inherited KerasRegressor class of keras.
    """

    def __init__(self, model_func):
        """
        Constructor.

        Prameters
        ---------
        model_func : function
            The function for creating model.
        """
        super().__init__(build_fn=model_func)

    def __getstate__(self):
        result = {'sk_params': self.sk_params}
        with tempfile.TemporaryDirectory() as dir:
            if hasattr(self, 'model'):
                self.model.save(dir + '/output.h5', include_optimizer=False)
                with open(dir + '/output.h5', 'rb') as f:
                    result['model'] = f.read()
        return result

    def __setstate__(self, serialized):
        self.sk_params = serialized['sk_params']
        with tempfile.TemporaryDirectory() as dir:
            model_data = serialized.get('model')
            if model_data:
                with open(dir + '/input.h5', 'wb') as f:
                    f.write(model_data)
                self.model = tf.keras.models.load_model(dir + '/input.h5')

    def fit(self, train_gen, val_gen, epochs, batch_size, callbacks):
        """
        Training model.

        Parameters
        ----------
        train_gen : iterator
            The generator of train data.
        val_gen : iterator
            The generator of validation data.
        epochs : int
            Number of epochs for training model.
        batch_size : int
            Size of batch for training model.
        callbacks : list
            The list of callbacks.
            For example [EarlyStopping instance, ModelCheckpoint instance]
        """
        self.model = self.build_fn
        self.model.fit(train_gen, epochs=epochs, batch_size=batch_size,
                       validation_data=val_gen, callbacks=callbacks)

    def evaluate(self, test_X, test_y):
        """
        Evaluate model.

        Parameters
        ----------
        test_X : iterator
            The generator of test data.
        test_y : np.ndarray
            The array of targets of test data.
        """
        pred = self.model.predict(test_X)
        pred = np.where(pred < 0, 0, pred)
        rmse = np.sqrt(mean_squared_error(test_y, pred))
        mae = np.sqrt(mean_absolute_error(test_y, pred))

        print(f"RMSE Score: {rmse}")
        print(f"MAE Score: {mae}")

    def predict(self, img_path: str, collection_name: str, num_sales: int,
                task: str = "B"):
        """
        Predict data using trained model.

        Parameters
        ----------
        img_path : str
            The path of image data.
        collection_name : str
            Name of collection of the NFT.
        num_sales : int
            Number of times the NFT sold.
        """
        if task == "A":
            collections = ['CryptoPunks',
                           'Bored Ape Yacht Club',
                           'Edifice by Ben Kovach',
                           'Mutant Ape Yacht Club',
                           'The Sandbox',
                           'Divine Anarchy',
                           'Cosmic Labs',
                           'Parallel Alpha',
                           'Art Wars | AW',
                           'Neo Tokyo Identities',
                           'Neo Tokyo Part 2 Vault Cards',
                           'Cool Cats NFT',
                           'CrypToadz by GREMPLIN',
                           'BearXLabs',
                           'Desperate ApeWives',
                           'Decentraland',
                           'Neo Tokyo Part 3 Item Caches',
                           'Doodles',
                           'The Doge Pound',
                           'Playboy Rabbitars Official',
                           'THE SHIBOSHIS',
                           'THE REAL GOAT SOCIETY',
                           'Sipherian Flash',
                           'Party Ape | Billionaire Club',
                           'Treeverse',
                           'Angry Apes United',
                           'CyberKongz',
                           'Emblem Vault [Ethereum]',
                           'Fat Ape Club',
                           'VeeFriends',
                           'JUNGLE FREAKS BY TROSLEY',
                           'Meebits',
                           'Furballs.com Official',
                           'Kaiju Kingz',
                           'Bears Deluxe',
                           'PUNKS Comic',
                           'Hor1zon Troopers',
                           'Lazy Lions',
                           'LOSTPOETS',
                           'Chain Runners',
                           'Chromie Squiggle by Snowfro',
                           'MekaVerse',
                           'Vox Collectibles',
                           'MutantCats',
                           'World of Women',
                           'SuperFarm Genesis Series',
                           'Eponym by ART AI',]
            collection_dict = {
                 collections[i]: i for i in range(len(collections))
            }
            meta_data = np.zeros(shape=(len(collection_dict)+1))
            if collection_name in collection_dict.keys():
                meta_data[collection_dict[collection_name]] = 1
            meta_data[-1] = num_sales
            meta_data = meta_data.reshape(1, -1)

            img = cv2.resize(cv2.imread(img_path)/256., (256, 256))
            img = img.reshape(1, 256, 256, 3)

            pred = self.model.predict([img, meta_data])
        elif task == "B":
            img = cv2.resize(cv2.imread(img_path)/256., (256, 256))
            img = img.reshape(1, 256, 256, 3)

            pred = self.model.predict(img)
        else:
            raise Exception("Please set task is A or B")

        return pred[0][0]

In [8]:
def save_model(instance, file_name: str):
    """
    Save model as pickle file

    Parameters
    ----------
    instance : Class instance
        The class instance you want to save as pickle file.
    file_name : str
        The absolute path of file saved the instance.
    """
    with open(file_name, mode='wb') as f:
        cloudpickle.dump(instance, f)

In [9]:
def load_model(file_name: str):
    """
    Load the model file of pickle.

    Parameters
    ----------
    file_name : str
        The absolute path of the model file.

    Returns
    -------
    model : tf.keras.models.Model
        Trained model object.
    """
    with open(file_name, mode='rb') as f:
        model = cloudpickle.load(f)

    return model

## Training models  

### TaskA

In [10]:
meta_features =\
    asset_df_A['collection.name'].unique().tolist() + ['num_sales']

path_list = asset_df_A['full_path'].values
meta_data = asset_df_A[meta_features].values
target = asset_df_A['target'].values

model_A = train(path_list, target, losses.mean_squared_error, meta_data,
                task="A")
# save_model(model_A, "../models/baselineA.pkl")

2021-11-14 08:31:09.150668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 08:31:09.155139: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 08:31:09.155625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 08:31:09.156568: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

starting training
*------------------------------*
Epoch 1/100


2021-11-14 08:31:10.735273: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-14 08:31:14.745896: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2021-11-14 08:31:16.235274: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1224/1224 [==============================] - 226s 180ms/step - loss: 6.9404 - mae: 1.2429 - mse: 6.9404 - val_loss: 5.3435 - val_mae: 1.0126 - val_mse: 5.3435
Epoch 2/100
1224/1224 [==============================] - 214s 174ms/step - loss: 3.5293 - mae: 0.9540 - mse: 3.5293 - val_loss: 2.9317 - val_mae: 0.9309 - val_mse: 2.9317
Epoch 3/100
1224/1224 [==============================] - 215s 176ms/step - loss: 1.8858 - mae: 0.8630 - mse: 1.8858 - val_loss: 2.2883 - val_mae: 0.9268 - val_mse: 2.2883
Epoch 4/100
1224/1224 [==============================] - 215s 175ms/step - loss: 1.5454 - mae: 0.8143 - mse: 1.5454 - val_loss: 2.1529 - val_mae: 0.9906 - val_mse: 2.1529
Epoch 5/100
1224/1224 [==============================] - 218s 178ms/step - loss: 1.3273 - mae: 0.7553 - mse: 1.3273 - val_loss: 40.6606 - val_mae: 4.4170 - val_mse: 40.6606
Epoch 6/100
1224/1224 [==============================] - 215s 176ms/step - loss: 1.2829 - mae: 0.7280 - mse: 1.2829 - val_loss: 4.8667 - val_mae: 1.2190 - 

### TaskA（画像のみ）  

In [11]:
path_list = asset_df_A['full_path'].values
target = asset_df_A['target'].values

model_A = train(path_list, target, losses.mean_squared_error,
                task="B")

starting training
*------------------------------*
Epoch 1/100
1224/1224 [==============================] - 217s 175ms/step - loss: 2.7349 - mae: 1.0369 - mse: 2.7349 - val_loss: 2.2998 - val_mae: 0.9054 - val_mse: 2.2998
Epoch 2/100
1224/1224 [==============================] - 211s 172ms/step - loss: 1.7788 - mae: 0.8671 - mse: 1.7788 - val_loss: 2.3431 - val_mae: 0.9206 - val_mse: 2.3431
Epoch 3/100
1224/1224 [==============================] - 213s 174ms/step - loss: 1.4670 - mae: 0.7940 - mse: 1.4670 - val_loss: 14.4225 - val_mae: 1.9665 - val_mse: 14.4225
Epoch 4/100
1224/1224 [==============================] - 211s 173ms/step - loss: 1.3555 - mae: 0.7492 - mse: 1.3555 - val_loss: 1.9551 - val_mae: 0.9155 - val_mse: 1.9551
Epoch 5/100
1224/1224 [==============================] - 211s 173ms/step - loss: 1.2367 - mae: 0.7087 - mse: 1.2367 - val_loss: 53.2000 - val_mae: 4.1408 - val_mse: 53.2000
Epoch 6/100
1224/1224 [==============================] - 212s 174ms/step - loss: 1.2245 - 

### TaskB

In [12]:
path_list = asset_df_B['full_path'].values
target = asset_df_B['target'].values

model_B = train(path_list, target, losses.mean_squared_error)
# save_model(model_B, "../models/baselineB.pkl")

starting training
*------------------------------*
Epoch 1/100
293/293 [==============================] - 57s 181ms/step - loss: 0.5716 - mae: 0.5127 - mse: 0.5716 - val_loss: 0.3407 - val_mae: 0.3353 - val_mse: 0.3407
Epoch 2/100
293/293 [==============================] - 52s 176ms/step - loss: 0.4381 - mae: 0.4247 - mse: 0.4381 - val_loss: 0.3661 - val_mae: 0.2986 - val_mse: 0.3661
Epoch 3/100
293/293 [==============================] - 52s 177ms/step - loss: 0.3878 - mae: 0.3914 - mse: 0.3878 - val_loss: 0.3510 - val_mae: 0.3146 - val_mse: 0.3510
Epoch 4/100
293/293 [==============================] - 51s 174ms/step - loss: 0.3690 - mae: 0.3748 - mse: 0.3690 - val_loss: 0.3481 - val_mae: 0.2835 - val_mse: 0.3481
Epoch 5/100
293/293 [==============================] - 51s 175ms/step - loss: 0.3544 - mae: 0.3763 - mse: 0.3544 - val_loss: 0.3389 - val_mae: 0.3086 - val_mse: 0.3389
Epoch 6/100
293/293 [==============================] - 51s 173ms/step - loss: 0.3485 - mae: 0.3657 - mse: 0.3

## Evaluate model  

### Task A

### Task B